In [12]:
###인터파크에서 요즘 HOT 콘서트에서 클로링해오기!!!!###
# 1, 싱글페이지 어플리케이션인지는 크롬확장프로그램으로 확인!(원자모양)
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
# 2,데이터를 전처리 한 후에 mysql로 보내기 위한 라이브러
import pandas as pd

service = Service(ChromeDriverManager().install()) #서비스 실행되게 세팅!

options = Options();
options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")
options.add_argument("--lang=ko_KR")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")

#실질적인 크롤링 역할
driver = webdriver.Chrome(service=service, options=options) #웹드라이버에서 각각의 역할을 하게 세팅

url = "https://tickets.interpark.com/contents/genre/concert"
driver.get(url) #url가서 크롤링해와
time.sleep(2) #2초동안 뜸을들여

# 3, 요즘HOT 이라는 버튼을 눌러야지 내가 원하는 값을 찾아올 수 있음!
try : 
    hot_button = driver.find_element(By.XPATH, "//button[contains(text(),'요즘 HOT')]")
    print(hot_button)#-> <selenium.webdriver.remote.webelement.WebElement (sess~~
    hot_button.click()
    time.sleep(2)
except Exception as e: #이 실행이 에러가 나면 except문을 실행시켜라!
    print("요즘 hot 버튼 클릭 실패 : ", e)

# 4, 스크롤한 높이를 기준으로 final높이로 맞춰주고 싶음
scroll_pause = 1 
last_height = driver.execute_script("return document.body.scrollHeight")

for _ in range(5):
    #반복할때마다 해당요소를 찾아옴
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #execute_script : 자바스크립트의 문안을 실행하자
    #5번을 행하는데 드라이버가 자바스크립트 문안을 통해 실행해 . 스크롤 할 수 있는 높이까지 스크롤해
    time.sleep(scroll_pause)
    #스크롤했는데, 다음 높이값이 더 추가되었을때 (보험용) 
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:#만약 새로운높이값이 나오면 마지막스크롤에 닿으면 멈춰라
        break
    last_height = new_height  #else문, else 생략. 
        #스크롤 할때마다 높이가 달라지는 페이지에서 쓰이는 문구!

# 5, 공연 제목, 지역, 공연장 요소값 찾아오기
titles = driver.find_elements(By.CSS_SELECTOR,".TicketItem_goodsName__Ju76j")
dates = driver.find_elements(By.CSS_SELECTOR,".TicketItem_playDate__5ePr2")
places= driver.find_elements(By.CSS_SELECTOR,".TicketItem_placeName__ls_9C")
#print(titles,dates,places) #-> 3묶음으로 리스트화 but 따로국밥!!
#하나의 레코드에는 ( title / date / place) ... 이렇게 출력되어야함.

# 6,
#전역변수로 딕셔너리로 모아놓은 리스트 하나 만들기
data = []

# for title in titles :
#     a = dict(title)
#     for date in dates:
#         a.values(date) -> 비효율적!!
#한번에 돌고 돌때마다 같은 인덱스에서 나온 값을 병렬로 직결시켜줄게! = zip
for t,d,p in zip(titles,dates,places):
    data.append({
        "title":t.text.strip(),
        "date":d.text.strip(),
        "place":p.text.strip()
    }) #딕셔너리 형태로 넣어놔야 행열로 쉽게 넣을 수 있음!

#print(data)

driver.quit()

# 7, SQL로 보내기
#어떠한데이터를 중심적으로 움직이는 지확인하는 판다스의 데이터프레임
df = pd.DataFrame(data)
df.head()
#이렇게 전처리를 마치면 SQL로 보내기 쉬워짐!!!! => 전처리끝!
# => MYSQL로 가서 데이터를 담을 데이터베이스 & 테이블에 담기! 250801 - SQL05

<selenium.webdriver.remote.webelement.WebElement (session="89026420fef70ae2a5d6c0577047e760", element="f.800E59356460B6E262EED40AC8CF0AB6.d.0E1BDC5B82260F1400765FD2AEAC647E.e.768")>


,title,date,place
0,변진섭 전국투어 콘서트 : 변천사 시즌2,2025.02.09 -,전국 각 지역
1,RETRO MUSIC CONCERT 〈최재훈×플라워〉,"2025.08.09, 2025.10.19","서울,인천"
2,옥탑재즈,2025.02.14 - 2025.10.24,아트센터인천 다목적홀
3,2025 심수봉 전국투어 콘서트：꽃길 - 안동,2025.09.13,안동문화예술의전당 웅부홀
4,홍경민 매달리스트 콘서트,2025.08.02,SA HALL


In [15]:
### 위의 셀은 데이터를 수집하는 셀!! ####
## 보내는 라이브러리 필요! ###
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [23]:
#파이썬에게 SQL로 보내라고 시키기!
import pymysql
# 1, 나의 신원정보를 확인하기 위한 host 정보값필요!
conn = pymysql.connect( #conn은 접속한 정보를 가져오는 객체!
    host = 'localhost',
    user = 'root',
    password = 'asdf1122',
    db = 'interparkmkt',
    charset ='utf8mb4', # 2, 언어인코딩 되는 것인지 알려주는 일종의 문법
)

# 3, conn: id와 패스워드로 접속 하고 DB안에 table값을 집어넣도록 하는 함수
cursor = conn.cursor()
# 4, cursor는 df 에 있는 프레임을 한줄씩 떼어서 SQL performance에 한줄씩 끼워넣어야함! 반복문으로
for _,row  in df.iterrows() : # iterrows : 추출해온 데이터에서 각각의 하나의 행을 반복적으로 찾아와라!
    # _ -> 각각의 행을 찾아올 때마다 인덱스 값을 찾아오는데 우리는 쓰지 않으니까 빈바_를 넣어줌
    sql = """ 
    INSERT INTO performance (title, date_range, place)
    VALUES (%s,%s,%s)
    """ # 따옴표안에 문장을 넣으면 그대로 다 문자로 인식함, sql문법을 넣으면 사용가능!, %s = 형식지정자
    cursor.execute(sql, (row["title"],row["date"],row["place"]))  
    # 위에 설정한 sql 변수를 sql에서 실행시키고
    # row는 title, date, place이라는 키값에 맞는 값을 가지고 있을것! 각각 %S로 위치에 맞게 들어갈 것!
    #이 값들을 각각 키에 맞게 넣어야한다!! -> course로!
    # 값을 전달한 후에 반영해야함!
conn.commit() #commit -> 확정 반영의 뜻. 지금까지 넣었던 값을 mysql에 반영해라!
conn.close()